In [1]:
import random 
import pickle

import numpy as np 
import pandas as pd 
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [2]:
text_df = pd.read_csv('data.csv')

In [3]:
text = list(text_df.text.values)
joined_text = " ".join(text)

In [4]:
partial_text = joined_text[:100000]

In [5]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [6]:
unique_tokens = np.unique(tokens)
unique_tokens_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [7]:
print(unique_tokens_index)

{'0': 0, '000': 1, '1': 2, '10': 3, '100': 4, '106': 5, '10th': 6, '11': 7, '12': 8, '12pm': 9, '13': 10, '14': 11, '15': 12, '16': 13, '160': 14, '17': 15, '187': 16, '19': 17, '1939': 18, '1960': 19, '197': 20, '1971': 21, '1972': 22, '1983': 23, '1984': 24, '1985': 25, '1990s': 26, '1992': 27, '1994': 28, '1996': 29, '1998': 30, '1999': 31, '2': 32, '20': 33, '200': 34, '2000': 35, '20001': 36, '2001': 37, '2002': 38, '2003': 39, '2004': 40, '2006': 41, '2007': 42, '2008': 43, '2009': 44, '2010': 45, '2011': 46, '2012': 47, '2013': 48, '2014': 49, '2015': 50, '2016': 51, '2020': 52, '21': 53, '2117': 54, '21st': 55, '21wire': 56, '22': 57, '227': 58, '23': 59, '24': 60, '241': 61, '27': 62, '2nd': 63, '3': 64, '30': 65, '300': 66, '31': 67, '33': 68, '3pm': 69, '4': 70, '40': 71, '400': 72, '42': 73, '49': 74, '5': 75, '50': 76, '500': 77, '51': 78, '52': 79, '57': 80, '5pm': 81, '6': 82, '60': 83, '64': 84, '650': 85, '7': 86, '70': 87, '73': 88, '8': 89, '80': 90, '8pm': 91, '9': 

In [8]:
n_words =  10 
input_words = []
next_words = []


for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_words.append (tokens[i + n_words])

In [9]:
input_words

[['daniel',
  'greenfield',
  'a',
  'shillman',
  'journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center'],
 ['greenfield',
  'a',
  'shillman',
  'journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is'],
 ['a',
  'shillman',
  'journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is',
  'a'],
 ['shillman',
  'journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is',
  'a',
  'new'],
 ['journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is',
  'a',
  'new',
  'york'],
 ['fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is',
  'a',
  'new',
  'york',
  'writer'],
 ['at',
  'the',
  'freedom',
  'center',
  'is',
  'a',
  'new',
  'york',
  'writer',
  'focusing'],
 ['the',
  'freedom',
  'center',
  'is',
  'a',
  'new',
  'york',
  'writer',
  'focusing',
  'on'],
 ['freedom',
  'center',
  'is',
  'a',
  'new',
  'york',
  'writer',
  'focusing',
  'on',
  'radical'],
 ['center',
  'is',
  'a',
  

In [10]:
next_words

['is',
 'a',
 'new',
 'york',
 'writer',
 'focusing',
 'on',
 'radical',
 'islam',
 'in',
 'the',
 'final',
 'stretch',
 'of',
 'the',
 'election',
 'hillary',
 'rodham',
 'clinton',
 'has',
 'gone',
 'to',
 'war',
 'with',
 'the',
 'fbi',
 'the',
 'word',
 'unprecedented',
 'has',
 'been',
 'thrown',
 'around',
 'so',
 'often',
 'this',
 'election',
 'that',
 'it',
 'ought',
 'to',
 'be',
 'retired',
 'but',
 'it',
 's',
 'still',
 'unprecedented',
 'for',
 'the',
 'nominee',
 'of',
 'a',
 'major',
 'political',
 'party',
 'to',
 'go',
 'war',
 'with',
 'the',
 'fbi',
 'but',
 'that',
 's',
 'exactly',
 'what',
 'hillary',
 'and',
 'her',
 'people',
 'have',
 'done',
 'coma',
 'patients',
 'just',
 'waking',
 'up',
 'now',
 'and',
 'watching',
 'an',
 'hour',
 'of',
 'cnn',
 'from',
 'their',
 'hospital',
 'beds',
 'would',
 'assume',
 'that',
 'fbi',
 'director',
 'james',
 'comey',
 'is',
 'hillary',
 's',
 'opponent',
 'in',
 'this',
 'election',
 'the',
 'fbi',
 'is',
 'under',
 '

In [11]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)



In [12]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_tokens_index[word]] = 1

    y[i, unique_tokens_index[next_words[i]]] = 1

In [13]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [14]:
model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.01), metrics=["accuracy"])
model.fit(X, y, batch_size=128, epochs=30, shuffle=True)

Epoch 1/30
135/135 [==============================] - 31s 195ms/step - loss: 6.8742 - accuracy: 0.0546
Epoch 2/30
135/135 [==============================] - 40s 296ms/step - loss: 6.6463 - accuracy: 0.0626
Epoch 3/30
135/135 [==============================] - 41s 307ms/step - loss: 6.5151 - accuracy: 0.0676
Epoch 4/30
135/135 [==============================] - 41s 303ms/step - loss: 6.3396 - accuracy: 0.0763
Epoch 5/30
135/135 [==============================] - 41s 303ms/step - loss: 6.1262 - accuracy: 0.0885
Epoch 6/30
135/135 [==============================] - 41s 303ms/step - loss: 5.8868 - accuracy: 0.1015
Epoch 7/30
135/135 [==============================] - 41s 301ms/step - loss: 5.6327 - accuracy: 0.1182
Epoch 8/30
135/135 [==============================] - 40s 296ms/step - loss: 5.3523 - accuracy: 0.1410
Epoch 9/30
135/135 [==============================] - 40s 297ms/step - loss: 5.0623 - accuracy: 0.1631
Epoch 10/30
135/135 [==============================] - 40s 297ms/step - l

In [15]:
model.save("mymodel.h5")

In [16]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1,n_words,len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_tokens_index[word]] = 1
    
    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [17]:
possible = predict_next_word('He will have to look into this thing and he', 5)

1/1 [==============================] - 1s 1s/step


In [18]:
print([unique_tokens[idx] for idx in possible])

['chairman', 'who', 'on', 'however', 'classroom']


In [19]:
def generate_text(input_text, text_length, creativity = 3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(text_length):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [20]:
generate_text("The Presidient of united states has announced that  ", 50, 1)

1/1 [==============================] - 0s 40ms/step


1/1 [==============================] - 0s 41ms/step


'The Presidient of united states has announced that from legislatures in for for of but we we we white they obama it the is it one has out one a hasn a hillary the the to was we nearly women in in i in i i baylee into i i my into in as found in in in'